### Animacy Polygon Segmentation Inter-Annotator Agreement

## Story 1


2023

This notebook imports the annotation data and provides the code to reproduce the results
for the animacy segmentation task. The results offer quantified measures
of inter-annotator agreement.

##### Libraries and packages

In [3]:

import json # json library
import os # for list directories 

import numpy as np 
import math
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

from collections import Counter # counter library
import re # regex

import itertools
from itertools import permutations # library to print permutations
from itertools import combinations # library to print combinations



##### Write-in JSON files

In [4]:

# Story 1 
f1 = open("Annotations_Prolific/Animacy_Story1/Processed/Round2/participant1.json")
f2 = open("Annotations_Prolific/Animacy_Story1/Processed/Round2/participant2.json")
f3 = open("Annotations_Prolific/Animacy_Story1/Processed/Round2/participant3.json")
f4 = open("Annotations_Prolific/Animacy_Story1/Processed/Round2/participant4.json")
f5 = open("Annotations_Prolific/Animacy_Story1/Processed/Round2/participant5.json")
f6 = open("Annotations_Prolific/Animacy_Story1/Processed/Round2/participant6.json")

participant1 = json.load(f1)
participant2 = json.load(f2)
participant3 = json.load(f3)
participant4 = json.load(f4)
participant5 = json.load(f5)
participant6 = json.load(f6)



##### Create dict containing all annotations per participant

In [5]:
# participant 2 is not included because this participant did not make any segmentations

story1 = {"participant1" : participant1,
          "participant3" : participant3,
          "participant4" : participant4,
          "participant5" : participant5,
          "participant6" : participant6
         }


### Polygon verification, IOU, and intersection functions

In [6]:

def get_polygon_coords_and_id_per_panel(story_annotations):
    """
    Takes: story_annotations - a dict holding one participants annotations for a story
    Returns: annotations_filtered_to_agent_segmentations - 
        {"page" : {"panel" : {"coords" : [coords]}, "animate" : {{"id1" : [coords]}, {"id2" : [coords]}... }}}
    """
    
    def convert_polygon_coords_to_list_of_tuples(coords):
        """
        Takes: list that holds dicts of coord pairs
        Returns: the same coords but a list of the coords as tuples
        """
        list_of_coord_tuples = []
        for coord in coords:
            list_of_coord_tuples.append((coord["x"], coord["y"]))
        return list_of_coord_tuples
        
    annotations_filtered_to_agent_segmentations = {}
    
    for page, page_info in story_annotations.items():
        annotations_filtered_to_agent_segmentations[page] = {} # add new page dict
        for panel, panel_info in page_info.items():
            panel_coords = panel_info["panelSize"]
            panel_coords_reformatted = convert_polygon_coords_to_list_of_tuples(panel_coords)
            annotations_filtered_to_agent_segmentations[page][panel] = {"coords" : panel_coords_reformatted} 
            agent_segmentations = panel_info["Agents"]
            annotations_filtered_to_agent_segmentations[page][panel]["animate"] = {}
            seg_coords_per_panel = {}
            for agent, agent_info in agent_segmentations.items():
                segmentation_coords = agent_info["agentSize"] # currently stored as a list of dicts
                segmentation_coords_reformatted = convert_polygon_coords_to_list_of_tuples(segmentation_coords)
                if len(segmentation_coords_reformatted) <= 2:
                    continue
                seg_coords_per_panel[agent_info["id"]] = segmentation_coords_reformatted
            annotations_filtered_to_agent_segmentations[page][panel]["animate"] = seg_coords_per_panel

    return annotations_filtered_to_agent_segmentations


In [7]:

def verify_polygons_and_iou(polygon_tuple1, polygon_tuple2):

    # convert to polygon objects
    polygon_object1 = Polygon(polygon_tuple1)
    polygon_object2 = Polygon(polygon_tuple2)

    # check that the polygon is closed
    if polygon_object1.is_valid == False:
        polygon_object1 = polygon_object1.buffer(0)
    if polygon_object2.is_valid == False:
        polygon_object2 = polygon_object2.buffer(0)

    intersect = polygon_object1.intersection(polygon_object2).area
    union = polygon_object1.union(polygon_object2).area
    iou = intersect / union
    
    return iou
        

In [8]:

def verify_polygons_and_intersection(polygon_tuple1, polygon_tuple2):
    
    # convert to polygon objects
    polygon_object1 = Polygon(polygon_tuple1)
    polygon_object2 = Polygon(polygon_tuple2)

    # check that the polygon is closed
    if polygon_object1.is_valid == False:
        polygon_object1 = polygon_object1.buffer(0)
    if polygon_object2.is_valid == False:
        polygon_object2 = polygon_object2.buffer(0)

    bool_contains = polygon_object1.contains(polygon_object2) # check that at least one point is overlapping between polygons
    intersect = polygon_object1.intersection(polygon_object2).area
    
    return bool_contains, intersect
    


### Dealing with large numbers of segmentations/permutations


In some panels, an annotator may have created a large number of segmentations (e.g. 10). Calculating IOUs between permutations of two sets of annotator's segments creates an intractable calculation. A function is therefore created to deal with this problem. In breif, all IOU scores between permutations of two sets of segments are put in a matrix. If a row and column in the matrix only has IOUs of zero, these are removed. Any row and associated column that has a value above zero contains segments that overlapped, and are used to calculate the final IOU scores. 

In [9]:

def matrix_filter(matrix):
    """
    This function finds matched segments, and finds which rows and columns
    to remove from large matrices
    Takes: a matrix that holds the iou scores between two annotators.
    Returns: 
        mapped_segments = lists of list pairs, with matched segements in each pair
        rows_to_filter = list of rows that can be filtered out of the matrix
        columns_to_filter = list of columns that can be filtered out of the matrix
        dummy_zero_matches = number of rows and col pairs with an iou of zero
    If a row has a unique value, 
    and if the associated column has a unique value, 
    this is a matched segment! Store this
    and keep track of the rows and columns that 
    are associated with this stored value
    """
    
    mapped_segments = []
    rows_to_filter = []
    columns_to_filter = []
    dummy_zero_matches = 0
    for r_index, r_values in enumerate(matrix):
        r_num_nonzero = 0
        r_index_nonzero = np.NaN
        for l_index, l_value in enumerate(r_values): # l_index, or list index, refers to columns
            if l_value != 0:
                r_num_nonzero += 1
                r_index_nonzero = l_index
        
        # if the row is all zeros:
        if r_num_nonzero == 0:
            c_index = 0 # if there is a column with only zeros, consider this a "dummy match" and throw them out
            for column in matrix.T:
                num_not_zeros = 0
                for i, v in enumerate(column):
                    if v != 0:
                        num_not_zeros += 1
                if num_not_zeros == 0 and c_index not in columns_to_filter:
                    columns_to_filter.append(c_index) 
                    rows_to_filter.append(r_index)
                    dummy_zero_matches += 1 # keep track of how many dummy matches are found
                    break
                c_index += 1
            continue
            
        # if there is a unique value in the row:
        if r_num_nonzero == 1: 
            c_values = [row[r_index_nonzero] for row in matrix] # get the column values for the row list index with a value
            c_num_nonzero = 0
            c_l_index_nonzero = np.NaN
            for c_l_index, c_l_value in enumerate(c_values):
                if c_l_index == r_index: # skip the unique value at the row index 
                    continue
                if c_l_value != 0:
                    c_num_nonzero += 1
                    c_l_index_nonzero = c_l_index
            if c_num_nonzero == 0: # there are no other unique values in the column
                mapped_segments.append([r_index, r_index_nonzero])
                columns_to_filter.append(r_index_nonzero)
                rows_to_filter.append(r_index)
            else:
                pass
        else:
            pass

    return mapped_segments, rows_to_filter, columns_to_filter, dummy_zero_matches




### Annotator agreement functions

In [10]:
# ANIMATE OUTLINE MAPPING WITH IOU SCORES:
# calculated ious between all combinations of animate segmentations per page between two annotators 
# returns overall iou average per panel and map of agreement between segmentations on that page

def calculate_ious_between_two_annotators(annotator1, annotator2):
    """
    The purpose of this function is to determine inter-annotator agreement between to annotators
        by calculating iou scores between each segmented polygon indicating animacy on each page. 
    Takes: two corpora of full annotatations, one corpus per annotator
    Returns:
        mapping_and_agreement is a dict holding all agreement stats per annotator pair:
        mapping_and_agreement[page][panel] = {"matched_segments" : matched_segments,
                                                  "best_perm" : best_perm_for_annotator_2,
                                                  "all_ious_in_best_perm" : best_perm_ious,
                                                  "mean_iou_inc_zeros" : iou_perm_average_inc_zeros,
                                                  "std_iou_inc_zeros" : iou_perm_std_inc_zeros,
                                                  "range_iou_inc_zeros" : iou_perm_range_inc_zeros,
                                                  "mean_iou_exc_zeros" : iou_perm_average_exc_zeros,
                                                  "std_iou_exc_zeros" : iou_perm_std_exc_zeros,
                                                  "range_iou_exc_zeros:" : iou_perm_range_exc_zeros}
    
    diff_num_segments_between_annotators = total difference in segmentations made between annotators, int  
    total_num_diff_segments_between_annotators = dict of differences in segmentations per panel, per page
    num_segs_ann1 = total number of segmentations made by annotator 1, int
    num_segs_ann2 = total number of segmentations made by annotator 2, int
    """
    
    TESTING = False   # set to False when ready for whole all against all setting
    
    filter_matrix = False # switch to turn on and off matrix filtering
    
    # filter the corpus to {"page" : {"panel" : {"coords" : [coords]}, "animate" : {{"id1" : [coords]}, {"id2" : [coords]}... }}}
    annotator1_coords = get_polygon_coords_and_id_per_panel(annotator1) 
    annotator2_coords = get_polygon_coords_and_id_per_panel(annotator2)

    # compare the intersection of every segmentation with every panel to determine which panel 
    # that each segmentation correctly belongs
    # Input structure - {"page" : {"panel" : {"coords" : [coords]}, "animate" : {{"id1" : [coords]}, {"id2" : [coords]}... }}}
    def segmentation_panel_correction(panel_and_animacy_segmentation_corpus):
        
        def dict_of_all_seg_coords_per_page(page_of_annotation):
            seg_coords_per_page=[]
            for panel, panel_info in page_of_annotation.items():
                animate_segmentations = panel_info["animate"]
                for seg, coords in animate_segmentations.items():
                    seg_name = panel + "_seg" + str(seg)
                    seg_coords_per_page.append({seg_name : coords})
            return seg_coords_per_page
        
        mapping_segments_to_panels = {}
        for page, page_info in panel_and_animacy_segmentation_corpus.items():
            mapping_segments_to_panels[page] = {}
            all_seg_coords_on_page = dict_of_all_seg_coords_per_page(panel_and_animacy_segmentation_corpus[page]) # list
            for panel, panel_info in page_info.items():
                mapping_segments_to_panels[page][panel] = {}
                scores_above_0 = {}
                panel_coords = panel_info["coords"]
                for id_seg_pair in all_seg_coords_on_page:
                    for seg_name, seg_coords in id_seg_pair.items():
                        current_contains, current_intersection = verify_polygons_and_intersection(panel_coords, seg_coords)
                        if current_intersection == 0 or current_intersection < 100.0: # the 100 px is negotiable...
                            continue
                        if current_intersection != 0:
                            scores_above_0[seg_name] = {"intersection_with_panel" : current_intersection,
                                                        "coords" : seg_coords}
                            best_intersection_score_per_panel = current_intersection
                            name_of_seg_with_best_score = seg_name
                mapping_segments_to_panels[page][panel] = scores_above_0
        return mapping_segments_to_panels
    
    
    # Returned structure - {page1 : {'panel1': {'panel1_seg1': {"intersection_with_panel" : float, "coords" : [coords]}, ...}}}
    panel_seg_mappings_annotator1 = segmentation_panel_correction(annotator1_coords)
    panel_seg_mappings_annotator2 = segmentation_panel_correction(annotator2_coords)
    
    # ensure the stories have the same number of polygon segmentations per page by adding dummy coords where needed
    diff_num_segments_between_annotators = {} # this is returned at the end of the function
    num_segs_ann1 = 0
    num_segs_ann2 = 0
    for page, page_info in panel_seg_mappings_annotator1.items():
        page_info_annotator2 = panel_seg_mappings_annotator2[page]
        diff_num_segments_between_annotators[page] = {}
        for panel, panel_info in page_info.items():
            panel_info_annotator2 = page_info_annotator2[panel]
            num_segs_ann1 = num_segs_ann1 + len(panel_info) # returned at the end of the function
            num_segs_ann2 = num_segs_ann2 + len(panel_info_annotator2) # returned at the end of the function
            diff_num_segments = len(panel_info) - len(panel_info_annotator2)
            if diff_num_segments != 0:
                diff_num_segments_between_annotators[page][panel] = {"total_diff" : abs(diff_num_segments),
                                                                     "ann1_total_segs" : len(panel_info),
                                                                     "ann2_total_segs" : len(panel_info_annotator2)}
            else:
                diff_num_segments_between_annotators[page][panel] = diff_num_segments
            if diff_num_segments < 0:
                diff = abs(diff_num_segments)
                for i in range(1, diff+1):
                    id_name = panel + "_dummy" + str(i)
                    panel_info[id_name] = {"intersection_with_panel" : 0.0,
                                           "coords" : [(0,0), (0,0), (0,0), (0,0)]}
            elif diff_num_segments > 0:
                diff = abs(diff_num_segments)
                for i in range(1, diff+1):
                    id_name = panel + "_dummy" + str(i)
                    panel_info_annotator2[id_name] = {"intersection_with_panel" : 0.0,
                                                      "coords" : [(0,0), (0,0), (0,0), (0,0)]}
            assert len(panel_info) == len(panel_info_annotator2)   
    # get total number of diff segments betwee annotators along with differences per page
    # these stats are returned at the end of the overall function
    total_num_diff_segments_between_annotators = 0
    print("diff segments between annotators: ", total_num_diff_segments_between_annotators)
    for page, page_info in diff_num_segments_between_annotators.items():
        for panel, num_segs in page_info.items():
            if num_segs != 0:
                total_num_diff_segments_between_annotators = total_num_diff_segments_between_annotators + num_segs["total_diff"]
            else:
                total_num_diff_segments_between_annotators  = total_num_diff_segments_between_annotators + num_segs
            
    # IOU agreement scores between annotator 1 and annotator 2
    # panel_seg_mappings_annotator1 = {page1 : {'panel1': {'panel1_seg1': {"intersection_with_panel" : float, "coords" : [coords]}, ...}}}
    mapping_and_agreement = {} # holds all mapping and agreement scores
    for page, panels_annotator1 in panel_seg_mappings_annotator1.items():
        mapping_and_agreement[page] = {}
        panels_annotator2 = panel_seg_mappings_annotator2[page]
        for panel, panel_info in panels_annotator1.items():
            mapping_and_agreement[page][panel] = {}
            panel_info_annotator2 = panels_annotator2[panel]
            # get the names and coords per seg, per panel, organised into two lists with corresponding indexes
            all_coords_ann1 = [] # note that this is all coords per panel in the annotation data, placed in a list
            all_names_ann1 = [] # note that this is all names per panel in the annotation data, placed in a list
            for seg_id, seg_info_ann1 in panel_info.items():
                current_coord_ann1 = seg_info_ann1["coords"]
                all_coords_ann1.append(current_coord_ann1)
                all_names_ann1.append(seg_id)
            assert len(all_coords_ann1) == len(all_names_ann1)
            all_coords_ann2 = []   
            all_names_ann2 = []
            for seg_id_ann2, seg_info_ann2 in panel_info_annotator2.items():
                current_coord_ann2 = seg_info_ann2["coords"]
                all_coords_ann2.append(current_coord_ann2)
                all_names_ann2.append(seg_id_ann2)
            assert len(all_coords_ann1) == len(all_names_ann1)
            assert len(all_coords_ann2) == len(all_names_ann2)
            
            # create a matrix between annotator 1 and annotator 2 segmentations and populate with the iou scores
            # each matrix is between the segmentations in each panel, rather than each page (to reduce computation time)
            all_ious_matrix = np.zeros([len(all_coords_ann1), len(all_coords_ann2)], dtype = float) 
            for i in range(0, len(all_coords_ann1)):
                for j in range(0, len(all_coords_ann2)):
                    iou_ann1_ann2 = verify_polygons_and_iou(all_coords_ann1[i], all_coords_ann2[j]) # get iou score
                    all_ious_matrix[i][j] = (iou_ann1_ann2) # i is ann1 row, j is ann2 col
               
            # check if the matrix needs filtering
            num_rows, num_cols = all_ious_matrix.shape
            if num_rows > 8:  # if there are more than 8 segments to check, filter the matrix
                filter_matrix = True
                filtered_mappings, remove_rows, remove_columns, dummy_matches = matrix_filter(all_ious_matrix) # all lists

                # get a list of remaining rows and cols
                remaining_rows = []
                remaining_cols = []
                for index in range(0,num_rows):
                    if index not in remove_rows:
                        remaining_rows.append(index)
                    if index not in remove_columns:
                        remaining_cols.append(index)

            # get the permutations to describe all possible segment pairs between annotators
            # then check which permutation has the highest iou average - this perm hold correctly matched segments
            best_perm_average = 0 # holds the best average iou indicator for the best perm
            best_perm_for_annotator_2 = None # holds the best perm
            best_perm_ious = [] # holds all iou scores in the best perm  
            
            if filter_matrix == True:
                segment_nums_ann1 = {}
                segment_nums_ann2 = {}
                for i_row, v_row in enumerate(remaining_rows):
                    segment_nums_ann1[i_row] = v_row
                for i_col, v_col in enumerate(remaining_cols):
                    segment_nums_ann2[i_col] = v_col
                perms = [x for x in itertools.permutations(list(range(0, len(remaining_rows))), len(remaining_rows))]
            else:
                perms = [x for x in itertools.permutations(list(range(0, len(all_coords_ann1))), len(all_coords_ann1))] # get all perms
            count = 0 # TESTING counter - just go through a subset of permutations to make sure everything is a-ok
            for perm in perms:
                count += 1  
                if TESTING and count > 100 and len(all_coords_ann1) > 5: # TEST 
                    print("breaking early for testing purposes, optimal scores not neccessarily found!")
                    break
                all_ious_in_perm = [] # list to hold all iou scores in current perm 
                for i, j in enumerate(list(perm)):
                    if filter_matrix == True: # a filtered matrix has indices that map to segment numbers on the overall matrix
                        i = segment_nums_ann1[i]
                        j = segment_nums_ann2[j]
                    current_polygon_iou = all_ious_matrix[i][j] # get previously calculated ious from matrix
                    all_ious_in_perm.append(current_polygon_iou)
                perm_average = np.mean(all_ious_in_perm)
                if perm_average > best_perm_average:
                    best_perm_average = perm_average 
                    best_perm_for_annotator_2 = perm # this is a tuple
                    best_perm_ious = tuple(all_ious_in_perm)
            best_perm_ious = list(best_perm_ious) # turn into a list 

            # filtered_mappings - list of list pairs, with ann1 in pair[0] and ann2 in pair[1] 
            if filter_matrix == True: # get the actual segment values for the best perm, and add matching segs
                if best_perm_for_annotator_2 != None: # if there is a best perm for ann 2,
                    temp_best_perm_annotator2 = [] # get seg values for ann2
                    for seg_key in best_perm_for_annotator_2:
                        temp_best_perm_annotator2.append(segment_nums_ann2[seg_key]) 
                    best_perm_for_annotator_2 = tuple(temp_best_perm_annotator2)

                    best_perm_for_annotator_1 = []
                    temp_best_perm_annotator1 = [] # get seg values for ann1
                    for index_ann1 in range(0, len(best_perm_for_annotator_2)):
                        temp_best_perm_annotator1.append(segment_nums_ann1[index_ann1])
                    best_perm_for_annotator_1 = tuple(temp_best_perm_annotator1)

                if len(filtered_mappings) != 0: # if there are some mappings,
                    best_perm_for_annotator_1 = [] # setup best perm for ann1,
                    best_perm_for_annotator_2 = [] # and change best perm ann2 to list from None
                    for map_pair in filtered_mappings: # add the mapped segments to their respective lists
                        best_perm_for_annotator_1.append(map_pair[0])
                        best_perm_for_annotator_2.append(map_pair[1])
                        # add the iou scores to the list of best perm ious
                        best_perm_ious.append(all_ious_matrix[map_pair[0]][map_pair[1]])   
                        
                if dummy_matches != 0: # if there are dummy matches - that is, pairs of 0s that have been filtered
                    for d in range(0, dummy_matches): # add a zero per match to the best perm iou list
                        best_perm_ious.append(0)
            
            
            # Stats of ious for best permutation for matched segments in each panel between annotators
            
            iou_perm_average_inc_zeros = np.mean(best_perm_ious)
            iou_perm_std_inc_zeros = np.std(best_perm_ious)
            if best_perm_ious == []: # if the panel is empty (there are no areas showing animacy at all for both annotators)
                iou_perm_range_inc_zeros = (np.nan, np.nan) # put the range as nan
            else:
                iou_perm_range_inc_zeros = (min(best_perm_ious), max(best_perm_ious))
            best_perm_ious_exc_zero = [x for x in filter(lambda x:x > 0, best_perm_ious)]
            iou_perm_average_exc_zeros = np.mean(best_perm_ious_exc_zero)
            iou_perm_std_exc_zeros = np.std(best_perm_ious_exc_zero)
            if best_perm_ious == []: # if the panel is empty (there are no areas showing animacy)
                iou_perm_range_exc_zeros = (np.nan, np.nan) # put the range as nan
            else:
                if best_perm_ious_exc_zero == []: # if the iou list is empty because all the ious were zero
                    iou_perm_range_exc_zeros = (np.nan, np.nan) # put the range as nan
                else:
                    iou_perm_range_exc_zeros = (min(best_perm_ious_exc_zero), max(best_perm_ious_exc_zero))
            
            # get the segment names of matched segments between ann1 and ann2 for the perm with the best average iou
            ann1_seg_names_in_best_perm = [] 
            ann2_seg_names_in_best_perm = [] 
            if best_perm_for_annotator_2 == None:
                ann1_seg_names_in_best_perm.append(None)
                ann2_seg_names_in_best_perm.append(None)
            elif filter_matrix == True: # gotta get the right names for ann1 if matrix is filtered
                for n in list(best_perm_for_annotator_1):
                    ann1_seg_names_in_best_perm.append("ann1: " + all_names_ann1[n])
                for m in list(best_perm_for_annotator_2):
                    ann2_seg_names_in_best_perm.append("ann2: " + all_names_ann2[m])
            else:
                for p, q in enumerate(list(best_perm_for_annotator_2)):
                    ann1_seg_names_in_best_perm.append("ann1: " + all_names_ann1[p])
                    ann2_seg_names_in_best_perm.append("ann2: " + all_names_ann2[q])
            matched_segments = list(zip(ann1_seg_names_in_best_perm, ann2_seg_names_in_best_perm))
            
            
            # put these results into the final mapping_and_agreement dict
            mapping_and_agreement[page][panel] = {"matched_segments" : matched_segments,
                                                  "best_perm" : best_perm_for_annotator_2,
                                                  "all_ious_in_best_perm" : best_perm_ious,
                                                  "mean_iou_inc_zeros" : iou_perm_average_inc_zeros,
                                                  "std_iou_inc_zeros" : iou_perm_std_inc_zeros,
                                                  "range_iou_inc_zeros" : iou_perm_range_inc_zeros,
                                                  "mean_iou_exc_zeros" : iou_perm_average_exc_zeros,
                                                  "std_iou_exc_zeros" : iou_perm_std_exc_zeros,
                                                  "range_iou_exc_zeros:" : iou_perm_range_exc_zeros}
            
            filter_matrix = False # turn off matrix filtering
            
    #print(mapping_and_agreement) # print here if you want to see all these agreements per panel
    
    return mapping_and_agreement, diff_num_segments_between_annotators, total_num_diff_segments_between_annotators, num_segs_ann1, num_segs_ann2
      
    
    

In [11]:
# mapping and agreements between two annotators structured as a dict
# this is the output of the previous function, and is the input of the function below
#
# mapping_and_agreement_dict structure = {"page" : {"panel" : {"matched_segments" : list of tuples,
#                                                   "best_perm" : tuple,
#                                                   "all_ious_in_best_perm" : list of floats,
#                                                   "mean_iou_inc_zeros" : float,
#                                                   "std_iou_inc_zeros" : float,
#                                                   "range_iou_inc_zeros" : tuple of floats,
#                                                   "mean_iou_exc_zeros" : float,
#                                                   "std_iou_exc_zeros" : float,
#                                                   "range_iou_exc_zeros:" : tuple of floats}

def overall_results(mapping_and_agreement_dict):
    """
    Takes: the mapping and agreement results dict returned by the calculate_ious_between_two_annotators function
    Results: 
        agreements_per_page = dict of agreement results per page between two annotators
        agreements_per_story = dict of agreement results per page between two annotators
    """
    all_mean_ious_per_story_inc_zeros = []
    all_mean_ious_per_story_NO_zeros = []
    agreements_per_page = {}
    for page, page_info in mapping_and_agreement_dict.items():
        all_mean_ious_per_page_inc_zeros = []
        all_mean_ious_per_page_NO_zeros = []
        for panel, panel_info in page_info.items():
            #print(panel_info["mean_iou_inc_zeros"])
            if math.isnan(panel_info["mean_iou_inc_zeros"]): # a nan means an empty panel, so remove these
                #print(panel)
                #print(panel_info["mean_iou_inc_zeros"])
                pass
            else:
                all_mean_ious_per_page_inc_zeros.append(panel_info["mean_iou_inc_zeros"])
                all_mean_ious_per_story_inc_zeros.append(panel_info["mean_iou_inc_zeros"])
            if math.isnan(panel_info["mean_iou_exc_zeros"]):
                pass
            else:
                all_mean_ious_per_page_NO_zeros.append(panel_info["mean_iou_exc_zeros"])
                all_mean_ious_per_story_NO_zeros.append(panel_info["mean_iou_exc_zeros"])
        agreements_per_page[page] = {"mean_iou_inc_zeros" : np.mean(all_mean_ious_per_page_inc_zeros),
                                     "std_iou_inc_zeros" : np.std(all_mean_ious_per_page_inc_zeros),
                                     "range_iou_inc_zeros" : (min(all_mean_ious_per_page_inc_zeros, default=0), max(all_mean_ious_per_page_inc_zeros, default=0)),
                                     "mean_iou_exc_zeros" : np.mean(all_mean_ious_per_page_NO_zeros),
                                     "std_iou_exc_zeros" : np.std(all_mean_ious_per_page_NO_zeros),
                                     "range_iou_exc_zeros" : (min(all_mean_ious_per_page_NO_zeros, default=0), max(all_mean_ious_per_page_NO_zeros, default=0))
                                     }
    agreements_per_story = {"mean_iou_inc_zeros" : np.mean(all_mean_ious_per_story_inc_zeros),
                            "std_iou_inc_zeros" : np.std(all_mean_ious_per_story_inc_zeros),
                            "range_iou_inc_zeros" : (min(all_mean_ious_per_story_inc_zeros, default=0), max(all_mean_ious_per_story_inc_zeros, default=0)),
                            "mean_iou_exc_zeros" : np.mean(all_mean_ious_per_story_NO_zeros),
                            "std_iou_exc_zeros" : np.std(all_mean_ious_per_story_NO_zeros),
                            "range_iou_exc_zeros" : (min(all_mean_ious_per_story_NO_zeros, default=0), max(all_mean_ious_per_story_NO_zeros, default=0))
                            }
    
    # print agreement s
    print("Agreements per Page: \n")
    print(agreements_per_page)
    print("\n")
    
    print("Agreements per Story: \n")
    print("Including zeros: \n")
    print("mean iou: ", np.mean(all_mean_ious_per_story_inc_zeros))
    print("std: ", np.std(all_mean_ious_per_story_inc_zeros))
    print("range: ", (min(all_mean_ious_per_story_inc_zeros, default=0), max(all_mean_ious_per_story_inc_zeros, default=0)), "\n")
    print("Excluding zeros: \n")
    print("mean iou: ", np.mean(all_mean_ious_per_story_NO_zeros))
    print("std: ", np.std(all_mean_ious_per_story_NO_zeros))
    print("range: ", (min(all_mean_ious_per_story_NO_zeros, default=0), max(all_mean_ious_per_story_NO_zeros, default=0)), "\n")
    
    return agreements_per_page, agreements_per_story
            
            
    
    

## Inter-Annotator Agreement Results

In [12]:

mean_ious_for_whole_story_inc_zeros = []
mean_ious_for_whole_story_NO_zeros = []
# get combination of all participants
print("STORY 1 - STUDY RESULTS \n") 
all_participants = ["participant1", "participant3", "participant4", "participant5", "participant6"]
combinations_of_participants = itertools.combinations(all_participants, 2)
for annotator_pairs in combinations_of_participants:
    print("~"*30, "\n")
    print("Agreement results between " + str(annotator_pairs[0]) + " and " + str(annotator_pairs[1]) + "\n")
    mapping_agreements, diff_num_segs, total_diff_num_segs, ann1_num_segs, ann2_num_segs  = calculate_ious_between_two_annotators(story1[annotator_pairs[0]], story1[annotator_pairs[1]])
    print("Difference in number of Segments per Panel: \n")
    print(diff_num_segs, "\n")
    print("Difference in number of Segments across the story: ", total_diff_num_segs, "\n")
    print(str(annotator_pairs[0]) + "'s total number of segmentations: ", ann1_num_segs, "\n")
    print(str(annotator_pairs[1]) + "'s total number of segmentations: ", ann2_num_segs, "\n")
    each_page_agreements, overall_story_agreements = overall_results(mapping_agreements)
    # Structure of overall_story_agreements
    # overall_story_agreements = {"mean_iou_inc_zeros" : float,
                                 #"std_iou_inc_zeros" : float,
                                 #"range_iou_inc_zeros" : tuple of floats,
                                 #"mean_iou_exc_zeros" : float,
                                 #"std_iou_exc_zeros" : float,
                                 #"range_iou_exc_zeros" : tuple of floats}
    mean_ious_for_whole_story_inc_zeros.append(overall_story_agreements["mean_iou_inc_zeros"])
    mean_ious_for_whole_story_NO_zeros.append(overall_story_agreements["mean_iou_exc_zeros"])

print("~"*30, "\n")

print("Overall story stats \n")

print("Number of participant combinations: ", len(mean_ious_for_whole_story_NO_zeros), "\n")

print("IOU scores per participant pair inc zeros: ")
print(mean_ious_for_whole_story_inc_zeros, "\n")

print("IOU scores per participant pair exc zeros: ")
print(mean_ious_for_whole_story_NO_zeros, "\n")

print("Stats Including Zeros: ")
print("Mean: ", np.mean(mean_ious_for_whole_story_inc_zeros))
print("Std: ", np.std(mean_ious_for_whole_story_inc_zeros))
print("Range: ", (min(mean_ious_for_whole_story_inc_zeros), max(mean_ious_for_whole_story_inc_zeros)), "\n")
print("Stats Excluding Zeros: ")
print("Mean iou exc zeros: ", np.mean(mean_ious_for_whole_story_NO_zeros))
print("Std: ", np.std(mean_ious_for_whole_story_NO_zeros))
print("Range: ", (min(mean_ious_for_whole_story_NO_zeros), max(mean_ious_for_whole_story_NO_zeros)), "\n")

print("Difference between zeros and no zeros mean: ", abs(np.mean(mean_ious_for_whole_story_inc_zeros)-np.mean(mean_ious_for_whole_story_NO_zeros)))






STORY 1 - STUDY RESULTS 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 

Agreement results between participant1 and participant3

diff segments between annotators:  0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered 

Difference in number of Segments per Panel: 

{'page1': {'panel1': {'total_diff': 1, 'ann1_total_segs': 18, 'ann2_total_segs': 17}, 'panel2': 0, 'panel3': 0}, 'page2': {'panel1': {'total_diff': 1, 'ann1_total_segs': 2, 'ann2_total_segs': 1}, 'panel2': {'total_diff': 1, 'ann1_total_segs': 2, 'ann2_total_segs': 1}, 'panel3': {'total_diff': 1, 'ann1_total_segs': 2, 'ann2_total_segs': 1}, 'panel4': {'total_diff': 1, 'ann1_total_segs': 10, 'ann2_total_segs': 9}, 'panel5': 0, 'panel6': 0, 'panel7': 0}, 'page3': {'panel1': {'total_diff': 2, 'ann1_total_segs': 9, 'ann2_total_segs': 7}, 'panel2': 0, 'panel3': 0, 'panel4': 0, 'panel5': {'total_diff': 1, 'ann1_total_segs': 5, 'ann2_total_segs': 4}, 'panel6': {'total_diff': 1, 'ann1_total_segs': 3, 'ann2_total_segs': 2}, 'panel7': 0}, 'page4': {'panel1': 0, 'panel2': 0, 'panel3': 0, 'panel4': 0, 'panel5': 0, 'panel6': {'total_diff': 1, 'ann1_total_segs': 8, 'ann2_total_segs': 7}, 'panel7': 0}, 'page5': {'panel1': {'total_diff': 1, 'ann1_total_segs

Difference in number of Segments per Panel: 

{'page1': {'panel1': {'total_diff': 16, 'ann1_total_segs': 18, 'ann2_total_segs': 2}, 'panel2': 0, 'panel3': 0}, 'page2': {'panel1': {'total_diff': 1, 'ann1_total_segs': 2, 'ann2_total_segs': 1}, 'panel2': 0, 'panel3': 0, 'panel4': {'total_diff': 10, 'ann1_total_segs': 10, 'ann2_total_segs': 0}, 'panel5': 0, 'panel6': 0, 'panel7': 0}, 'page3': {'panel1': {'total_diff': 7, 'ann1_total_segs': 9, 'ann2_total_segs': 2}, 'panel2': {'total_diff': 1, 'ann1_total_segs': 3, 'ann2_total_segs': 2}, 'panel3': 0, 'panel4': 0, 'panel5': {'total_diff': 1, 'ann1_total_segs': 5, 'ann2_total_segs': 4}, 'panel6': {'total_diff': 1, 'ann1_total_segs': 3, 'ann2_total_segs': 2}, 'panel7': 0}, 'page4': {'panel1': 0, 'panel2': 0, 'panel3': 0, 'panel4': 0, 'panel5': 0, 'panel6': {'total_diff': 2, 'ann1_total_segs': 8, 'ann2_total_segs': 6}, 'panel7': 0}, 'page5': {'panel1': {'total_diff': 1, 'ann1_total_segs': 2, 'ann2_total_segs': 1}, 'panel2': 0, 'panel3': 0, 'pan

diff segments between annotators:  0
Difference in number of Segments per Panel: 

{'page1': {'panel1': {'total_diff': 15, 'ann1_total_segs': 17, 'ann2_total_segs': 2}, 'panel2': 0, 'panel3': 0}, 'page2': {'panel1': 0, 'panel2': {'total_diff': 1, 'ann1_total_segs': 1, 'ann2_total_segs': 2}, 'panel3': {'total_diff': 1, 'ann1_total_segs': 1, 'ann2_total_segs': 2}, 'panel4': {'total_diff': 9, 'ann1_total_segs': 9, 'ann2_total_segs': 0}, 'panel5': 0, 'panel6': 0, 'panel7': 0}, 'page3': {'panel1': {'total_diff': 5, 'ann1_total_segs': 7, 'ann2_total_segs': 2}, 'panel2': {'total_diff': 1, 'ann1_total_segs': 3, 'ann2_total_segs': 2}, 'panel3': 0, 'panel4': 0, 'panel5': 0, 'panel6': 0, 'panel7': 0}, 'page4': {'panel1': 0, 'panel2': 0, 'panel3': 0, 'panel4': 0, 'panel5': 0, 'panel6': {'total_diff': 1, 'ann1_total_segs': 7, 'ann2_total_segs': 6}, 'panel7': 0}, 'page5': {'panel1': 0, 'panel2': 0, 'panel3': 0, 'panel4': 0, 'panel5': 0, 'panel6': {'total_diff': 8, 'ann1_total_segs': 8, 'ann2_total_s

Difference in number of Segments per Panel: 

{'page1': {'panel1': {'total_diff': 16, 'ann1_total_segs': 18, 'ann2_total_segs': 2}, 'panel2': 0, 'panel3': 0}, 'page2': {'panel1': {'total_diff': 1, 'ann1_total_segs': 2, 'ann2_total_segs': 1}, 'panel2': 0, 'panel3': 0, 'panel4': {'total_diff': 10, 'ann1_total_segs': 10, 'ann2_total_segs': 0}, 'panel5': 0, 'panel6': 0, 'panel7': 0}, 'page3': {'panel1': {'total_diff': 7, 'ann1_total_segs': 9, 'ann2_total_segs': 2}, 'panel2': {'total_diff': 1, 'ann1_total_segs': 3, 'ann2_total_segs': 2}, 'panel3': 0, 'panel4': {'total_diff': 1, 'ann1_total_segs': 6, 'ann2_total_segs': 5}, 'panel5': 0, 'panel6': 0, 'panel7': 0}, 'page4': {'panel1': 0, 'panel2': 0, 'panel3': 0, 'panel4': 0, 'panel5': 0, 'panel6': {'total_diff': 1, 'ann1_total_segs': 7, 'ann2_total_segs': 6}, 'panel7': 0}, 'page5': {'panel1': 0, 'panel2': 0, 'panel3': 0, 'panel4': 0, 'panel5': 0, 'panel6': {'total_diff': 8, 'ann1_total_segs': 8, 'ann2_total_segs': 0}, 'panel7': 0, 'panel8': 0}